# Homework 5

## by Dion Ho


# Import Python Libraries

In [1]:
import numpy as np
import math
import scipy
from scipy import linalg as la
from scipy import sparse
from math import pi
from math import factorial
from matplotlib import pyplot as plt

In [2]:
def rangeE(start,end): #I really dislike Python's range function, so I defined my own in line with other languages.
    return range(start,end+1)

## Q1

Since $\lVert\vec{x}\rVert_\infty = 1, \forall x_j \in \vec{x}, -1 \leq \vec{x_j} \leq 1$.
Each entry, $b_i$, of vector $(A\vec{x})$ is given by
$$b_i = \sum_{j=1}^n x_j a_{i,j}.$$
In order to maximise $b_i$, we choose
$$\begin{cases}
x_j = 1 & \mbox{if } a_{i,j} \geq 0 \\
x_j = -1 & \mbox{if } a_{i,j} < 0.
\end{cases}
$$
Therefore, 
$$\max\limits_{\lVert\vec{x}\rVert_\infty = 1}b_i = \sum_{j=1}^n |a_{i,j}|$$
which implies that 
\begin{align}
\lVert A \rVert_\infty &= \max\limits_{\lVert\vec{x}\rVert_\infty = 1} \lVert A\vec{x} \rVert_\infty \\
&= \max\limits_{1 \leq i \leq n} \sum_{j=1}^n |a_{i,j}|.
\end{align}

## Q2

Since the sequence $\left (\vec{x}^{\left (k\right )}\right )_{k \in \mathbb{N}^0}$ converges, it must be Cauchy. Therefore, for an arbitrary $\vec{x}^{\left (0\right )} \in \mathbb{R}^n$, for all $\epsilon z > 0$, where $z = \left \lVert \vec{x}^{\left (1\right )} - \vec{x}^{\left (0\right )} \right \rVert, \exists K \in \mathbb{N}^0$ such that $\forall m,k \geq K$, 
$$\left \lVert \vec{x}^{\left (m\right )} - \vec{x}^{\left (k\right )} \right \rVert< \epsilon z.$$
In particular, $\forall k \geq K$,
\begin{align}
&\left \lVert \vec{x}^{\left (k+1\right )} - \vec{x}^{\left (k\right )} \right \rVert < \epsilon z \\
&\implies \left \lVert T\vec{x}^{\left (k\right )} + \vec{c} - T\vec{x}^{\left (k-1\right )} - \vec{c} \right \rVert < \epsilon z \\
&\implies \left \lVert T\left (\vec{x}^{\left (k\right )} - \vec{x}^{\left (k-1\right )}\right ) \right \rVert < \epsilon z \\
&\implies \left \lVert T\left (T\vec{x}^{\left (k-1\right )} + \vec{c} - T\vec{x}^{\left (k-2\right )} - \vec{c}\right ) \right \rVert < \epsilon z \\
&\implies \left \lVert T^2\left (\vec{x}^{\left (k-1\right )} - \vec{x}^{\left (k-2\right )}\right ) \right \rVert < \epsilon z \\
&\implies \ldots \\
&\implies \left \lVert T^k \left (\vec{x}^{\left (1\right )} - \vec{x}^{\left (0\right )}\right ) \right \rVert < \epsilon z \\
&\implies \left \lVert T^k \right \rVert \left \lVert \vec{x}^{\left (1\right )} - \vec{x}^{\left (0\right )} \right \rVert \leq \left \lVert T^k \left (\vec{x}^{\left (1\right )} - \vec{x}^{\left (0\right )}\right ) \right \rVert < \epsilon z \\
&\implies \left \lVert T^k \right \rVert < \epsilon.
\end{align}
Therefore, $\lim\limits_{k\rightarrow\infty}T^k = 0_{nxn}$ which implies that $\rho\left (T\right ) < 1$ by property of the spectral radius.

## Q3

In [3]:
def jacobian_iteration(A,b,x1,tol=1e-8,maxiter=100):
    if A.shape[0] != A.shape[1]:
        return "Error, matrix is not square."
    m = A.shape[1]
    b.shape = (m,)
    x1.shape = (m,)
    D = np.diag(np.diag(A))
    AA = A - D
    D_inv = np.diag(1/np.diag(A))
    x = D_inv @ (b - (AA @ x1))
    iteration = 1
    residue = la.norm((x1 - x),np.inf)
    print("iter","   residue")
    print('{:>3d} {:> 22.16f}'.format(iteration, residue))
    while (residue > tol) & (maxiter >= iteration):
        x1 = x
        x = D_inv @ (b - (AA @ x1))
        residue = la.norm((x1 - x),np.inf)
        iteration += 1
        print('{:>3d} {:> 22.16f}'.format(iteration, residue))
    if maxiter >= iteration + 1:
        return x
    else:
        print("Failed to converge.")

In [4]:
def diag_dom(n, num_entries=None):
    if num_entries is None:
        num_entries = int(n**1.5) - n
    A = np.zeros((n,n))
    rows = np.random.choice(np.arange(0,n), size=num_entries)
    cols = np.random.choice(np.arange(0,n), size=num_entries)
    data = np.random.randint(-4, 4, size=num_entries)
    for i in range(num_entries):
        A[rows[i], cols[i]] = data[i]
    for i in range(n):
        A[i,i] = np.sum(np.abs(A[i])) + 1
    return A

Code to generate the randomised test data.

In [5]:
size = np.random.randint(2,20)
x = np.random.rand(size,1)
A = diag_dom(size)
b = A @ x
x1 = np.random.rand(size,1)

In [7]:
ans = jacobian_iteration(A,b,x1)
print(ans,np.allclose(ans.reshape(len(ans),), x.reshape(len(x),))) #jacobian_iteration works even for random A, x, and initial guess x1.

iter    residue
  1     0.3189834692606852
  2     0.0000000000000000
[0.16963229 0.74495377 0.20145741] True


## Q4

In [ ]:
def jacobian_iteration_plot(A,b,x1,plot = False,tol=1e-8,maxiter=100):
    if A.shape[0] != A.shape[1]:
        return "Error, matrix is not square."
    m = A.shape[1]
    b.shape = (m,)
    x1.shape = (m,)
    D = np.diag(np.diag(A))
    AA = A - D
    D_inv = np.diag(1/np.diag(A))
    x = D_inv @ (b - (AA @ x1))
    iteration = 1
    residue = la.norm((x1 - x),np.inf)
    if plot == True:
        res_plt = [residue]
    else:
        print("iter","   residue")
        print('{:>3d} {:> 22.16f}'.format(iteration, residue))
    while (residue > tol) & (maxiter >= iteration):
        x1 = x
        x = D_inv @ (b - (AA @ x1))
        residue = la.norm((x1 - x),np.inf)
        iteration += 1
        if plot == True:
            res_plt.append(residue)
        else:
            print('{:>3d} {:> 22.16f}'.format(iteration, residue))
    if plot == True:
        plt.figure(figsize=(10,5))
        plt.semilogy(rangeE(1,iteration),np.abs(res_plt),'o')
        plt.title('Plot of Residue for Jacobian Iteration against Iteration Count',fontsize=18)
        plt.xlabel('Iteration Count',fontsize=14)
        plt.ylabel('Residue',fontsize=14)
    if maxiter >= iteration + 1:
        return x
    else:
        print("Failed to converge.")

Code to generate the randomised test data.

In [ ]:
size = np.random.randint(3,20)
x = np.random.rand(size,1)
A = diag_dom(size)
b = A @ x
x1 = np.random.rand(size,1)

In [ ]:
ans = jacobian_iteration_plot(A,b,x1,True)
print(ans,np.allclose(ans, x)) #The plot shows that the log of the residue decreases about linearly as the number of iterations increase.

## Q5: With the addition of tuning parameter $\omega$ which defaults to 1.

In [ ]:
def gauss_seidel_plot(A,b,x1,plot=False,tol=1e-8,maxiter=100,omega=1):
    if A.shape[0] != A.shape[1]:
        return "Error, matrix is not square."
    m = A.shape[1]
    b.shape = (m,)
    x1.shape = (m,)
    x = np.zeros(np.shape(b))
    for ii in rangeE(0,m-1):
        sum1 = 0
        sum2 = 0
        for jj in rangeE(0,ii-1):
            sum1 += A[ii,jj]*x[jj]
        for jj in rangeE(ii,m-1): #Without omega, this is rangeE(ii+1,m-1)
            sum2 += A[ii,jj]*x1[jj]
        x[ii] = x1[ii] + omega*(b[ii] - sum1 - sum2)/A[ii,ii]
    iteration = 1
    residue = la.norm((x1 - x),np.inf)
    if plot == True:
        res_plt = [residue]
    else:
        print("iter","   residue")
        print('{:>3d} {:> 22.16f}'.format(iteration, residue))
    while (residue > tol) & (maxiter >= iteration):
        x1 = np.copy(x)
        for ii in rangeE(0,m-1):
            sum1 = 0
            sum2 = 0
            for jj in rangeE(0,ii-1):
                sum1 += A[ii,jj]*x[jj]
            for jj in rangeE(ii,m-1):
                sum2 += A[ii,jj]*x1[jj]
            x[ii] = x1[ii] + omega*(b[ii] - sum1 - sum2)/A[ii,ii]
        residue = la.norm((x1 - x),np.inf)
        iteration += 1
        if plot == True:
            res_plt.append(residue)
        else:
            print('{:>3d} {:> 22.16f}'.format(iteration, residue))
    if plot == True:
        plt.figure(figsize=(10,5))
        plt.semilogy(rangeE(1,iteration),np.abs(res_plt),'o')
        plt.title('Plot of Residue for Gauss-Seidel Method against Iteration Count',fontsize=18)
        plt.xlabel('Iteration Count',fontsize=14)
        plt.ylabel('Residue',fontsize=14)
    if maxiter >= iteration + 1:
        return x
    else:
        print("Failed to converge.")

Code to generate the randomised test data.

In [ ]:
size = np.random.randint(3,20)
x = np.random.rand(size,1)
A = diag_dom(size)
b = A @ x
x1 = np.random.rand(size,1)

In [ ]:
ans = gauss_seidel_plot(A,b,x1,True)
print(ans,np.allclose(ans, x)) #The plot shows that the log of the residue decreases about linearly as the number of iterations increase.

## Q6 & Q7: Gauss-Seidel with the addition of tuning parameter $\omega$ which defaults to 1.

In [16]:
def gauss_seidel_plot_sparse(A,b,x1,plot=False,tol=1e-8,maxiter=100,omega=1):
    if A.shape[0] != A.shape[1]:
        return "Error, matrix is not square."
    m = A.shape[1]
    b.shape = (m,)
    x1.shape = (m,)
    x = np.zeros(np.shape(b))
    D = sparse.diags(A.diagonal())
    AA = A - D
    for ii in rangeE(0,m-1):
        rowstart = AA.indptr[ii]
        rowend = AA.indptr[ii+1]
        Aix = AA.data[rowstart:rowend] @ x1[A.indices[rowstart:rowend]]
        x[ii] = (1-omega)*x1[ii] + omega*(b[ii] - Aix)/A[ii,ii]
    iteration = 1
    residue = la.norm((x1 - x),np.inf)
    if plot == True:
        res_plt = [residue]
    else:
        print("iter","   residue")
        print('{:>3d} {:> 22.16f}'.format(iteration, residue))
    while (residue > tol) & (maxiter >= iteration):
        x1 = np.copy(x)
        for ii in rangeE(0,m-1):
            rowstart = AA.indptr[ii]
            rowend = AA.indptr[ii+1]
            Aix = AA.data[rowstart:rowend] @ x1[AA.indices[rowstart:rowend]]
            x[ii] = (1-omega)*x1[ii] + omega*(b[ii] - Aix)/A[ii,ii]
        residue = la.norm((x1 - x),np.inf)
        iteration += 1
        if plot == True:
            res_plt.append(residue)
        else:
            print('{:>3d} {:> 22.16f}'.format(iteration, residue))
    if plot == True:
        plt.figure(figsize=(10,5))
        plt.semilogy(rangeE(1,iteration),np.abs(res_plt),'o')
        plt.title('Plot of Residue for Gauss-Seidel Method against Iteration Count',fontsize=18)
        plt.xlabel('Iteration Count',fontsize=14)
        plt.ylabel('Residue',fontsize=14)
    if maxiter >= iteration + 1:
        return x
    else:
        print("Failed to converge.")

In [9]:
def diag_dom_sparse(n, num_entries=None):
    if num_entries is None:
        num_entries = int(n**1.5) - n
    rows = np.random.choice(np.arange(0,n), size=num_entries)
    cols = np.random.choice(np.arange(0,n), size=num_entries)
    data = np.random.randint(-4, 4, size=num_entries)
    A = sparse.csr_matrix((data, (rows,cols)), shape=(n,n))
    d = []
    for i in range(n):
        d.append(np.sum(np.abs(A[i])) + 1)
    A.setdiag(d)
    return A

In [10]:
size = np.random.randint(5000,15000)
x = np.random.rand(size,1)
x.shape=(size,)
A = diag_dom_sparse(size) #Generation of the sparse matrix may take about a minute.
b = A @ x
x1 = np.random.rand(size,1)

c:\users\dionh\appdata\local\programs\python\python36\lib\site-packages\scipy\sparse\compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


In [23]:
ans = gauss_seidel_plot_sparse(A,b,x1,omega=1.5)
print(ans,np.allclose(ans,x)) #I only ran a few tests, but it seems like omega = 1 frequently results in the fastest convergence.

iter    residue
  1     1.6211828644261368
  2     1.0814543409812978
  3     0.6682127877062565
  4     0.4138431535906028
  5     0.2932435270609431
  6     0.1983766915404211
  7     0.1292259979360931
  8     0.0914851444299374
  9     0.0632694287006306
 10     0.0426529501929498
 11     0.0282356594854396
 12     0.0189448850996428
 13     0.0129344234079372
 14     0.0087190471817860
 15     0.0058890032482547
 16     0.0039644293823403
 17     0.0026488894873469
 18     0.0018213729583426
 19     0.0012413264062306
 20     0.0008494353840393
 21     0.0005857600445137
 22     0.0003993364231445
 23     0.0002752790600370
 24     0.0001916607769230
 25     0.0001322322792952
 26     0.0000904934860086
 27     0.0000616627177045
 28     0.0000427639600682
 29     0.0000293722901618
 30     0.0000199896277912
 31     0.0000134851735635
 32     0.0000090208437430
 33     0.0000059855910045
 34     0.0000040338983837
 35     0.0000028358688050
 36     0.0000019850337604
 37     0.00